# Song Proxy Re-encryption demo 
### via Umbral & Pinata IPFS Services

In [ ]:
# !pip install umbral cryptography ipfshttpclient
# !pip install cryptography ipfshttpclient

## Pinata API

In [ ]:
# curl --request GET \
#      --url https://api.pinata.cloud/data/testAuthentication \
#      --header 'accept: application/json' \
#      --header 'authorization: Bearer 'your JWT token'

In [ ]:
# {"message":"Congratulations! You are communicating with the Pinata API!"}%  

In [12]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os
import ipfshttpclient

# === Step 1: Generate a random AES key ===
aes_key = os.urandom(32)  # AES-256
iv = os.urandom(16)       # Initialization Vector

# === Step 2: Read and encrypt the MP3 file ===
file_path = '/Users/thanakrit/Documents/SoundRise_new/music files/japanese_fish.mp3'  # Replace with your local MP3 file
with open(file_path, 'rb') as f:
    mp3_data = f.read()

# Pad the MP3 data for AES block alignment
padder = padding.PKCS7(128).padder()
padded_data = padder.update(mp3_data) + padder.finalize()

# Encrypt
cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded_data) + encryptor.finalize()

# === Step 3: Save encrypted file locally ===
with open('encrypted_audio.bin', 'wb') as f:
    f.write(iv + ciphertext)  # prepend IV for decryption later

In [ ]:
import requests

PINATA_JWT = "ํYour pinata jwt"  # Get this from your Pinata dashboard

headers = {
    "Authorization": f"Bearer {PINATA_JWT}"
}

with open("encrypted_audio.bin", "rb") as f:
    files = {
        "file": ("encrypted_audio_2.bin", f)
    }

    response = requests.post(
        "https://api.pinata.cloud/pinning/pinFileToIPFS",
        files=files,
        headers=headers
    )

    print(response.json())

{'IpfsHash': 'QmZpJnBR1sN8cXgWfJQoKtKtGnLA9MDKZYqBDpkPVMfL5L', 'PinSize': 1067134, 'Timestamp': '2025-03-29T16:49:35.801Z', 'ID': 'ca2b1c3d-a85d-4e2e-a887-4fa3a57aca7f', 'Name': 'encrypted_audio_2.bin', 'NumberOfFiles': 1, 'MimeType': 'application/octet-stream', 'GroupId': None, 'Keyvalues': None}


## Try to combine with encryption stuff

In [15]:
from umbral import SecretKey, Signer, encrypt

# === Step 1: Generate Umbral key pairs ===
# Alice's keys
alices_secret_key = SecretKey.random()
alices_public_key = alices_secret_key.public_key()

alices_signing_key = SecretKey.random()
alices_signer = Signer(alices_signing_key)
alices_verifying_key = alices_signing_key.public_key()

# Bob's keys
bobs_secret_key = SecretKey.random()
bobs_public_key = bobs_secret_key.public_key()

# === Step 2: Encrypt the AES key using Alice's public key ===
capsule, encrypted_aes_key = encrypt(alices_public_key, aes_key)

# # Save capsule and encrypted key (optional for reuse or sharing)
# import pickle
# with open("capsule.bin", "wb") as f:
#     pickle.dump(capsule, f)

# with open("encrypted_aes_key.bin", "wb") as f:
#     f.write(encrypted_aes_key)

# print("AES key encrypted and capsule saved.")

In [16]:
from umbral import generate_kfrags

# Alice generates "M of N" re-encryption key fragments for Bob
# For example: 10-of-20 (Bob needs any 10 out of 20)
kfrags = generate_kfrags(
    delegating_sk=alices_secret_key,
    receiving_pk=bobs_public_key,
    signer=alices_signer,
    threshold=10,  # minimum number of cfrags needed
    shares=20      # total number of kfrags generated
)

print(f"{len(kfrags)} kfrags generated.")

20 kfrags generated.


In [17]:
from umbral import reencrypt
from umbral import CapsuleFrag  # Just for type clarity

# Simulate proxy re-encrypting using kfrags (pick 10, threshold)
selected_kfrags = kfrags[:10]  # We just use the first 10 for simplicity

cfrags = []  # Re-encrypted capsule fragments
for kfrag in selected_kfrags:
    cfrag = reencrypt(kfrag=kfrag, capsule=capsule)
    cfrags.append(cfrag)

# # Attach the cfrags to the capsule so Bob can use them
# for cfrag in cfrags:
#     capsule.attach_cfrag(cfrag)

print(f"{len(cfrags)} cfrags attached to the capsule.")

10 cfrags attached to the capsule.


In [18]:
from umbral import pre

# Bob decrypts the AES key using the re-encrypted capsule and cfrags
bob_aes_key = pre.decrypt_reencrypted(
    receiving_sk=bobs_secret_key,        # Bob’s private key
    delegating_pk=alices_public_key,     # Alice’s public key (used during original encryption)
    capsule=capsule,                     # Capsule with attached cfrags
    verified_cfrags=cfrags,              # The re-encrypted fragments from proxies
    ciphertext=encrypted_aes_key         # The encrypted AES key
)

assert bob_aes_key == aes_key  # Sanity check!
print("✅ Bob successfully recovered the AES key!")

✅ Bob successfully recovered the AES key!


In [19]:
aes_key

b'5\x9ak\x89\t\x1cXI\xf4\x82]\xbf\xa9\xe9\xe9\xf5\xa7\x07\x91\xf6f\xf1\x90!\xef\xb2\xd9\x06]\xbf`&'

## Trying to download from pinata

In [21]:
#cid is response.json()['IpfsHash']

cid = response.json()['IpfsHash']
cid

'QmZpJnBR1sN8cXgWfJQoKtKtGnLA9MDKZYqBDpkPVMfL5L'

In [22]:
import requests

# # Replace with your actual CID from upload step
# cid = "QmYourEncryptedFileHashHere"

# Default Pinata public gateway
url = f"https://gateway.pinata.cloud/ipfs/{cid}"

# Or use your custom gateway:
# url = f"https://your-gateway-name.mypinata.cloud/ipfs/{cid}"

response = requests.get(url)

if response.status_code == 200:
    with open("downloaded_encrypted_audio.bin", "wb") as f:
        f.write(response.content)
    print("✅ File downloaded from IPFS via Pinata.")
else:
    print("❌ Failed to download:", response.status_code, response.text)

✅ File downloaded from IPFS via Pinata.


In [23]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend

# === Step 1: Read the encrypted file ===
with open("/Users/thanakrit/Documents/SoundRise_new/downloaded_encrypted_audio.bin", "rb") as f:
    encrypted_data = f.read()

# Extract IV and ciphertext
iv = encrypted_data[:16]
ciphertext = encrypted_data[16:]

# === Step 2: AES decryption ===
cipher = Cipher(algorithms.AES(bob_aes_key), modes.CBC(iv), backend=default_backend())
decryptor = cipher.decryptor()
padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()

# === Step 3: Remove padding ===
unpadder = padding.PKCS7(128).unpadder()
mp3_data = unpadder.update(padded_plaintext) + unpadder.finalize()

# === Step 4: Save decrypted MP3 ===
with open("/Users/thanakrit/Documents/SoundRise_new/decrypted_audio.mp3", "wb") as f:
    f.write(mp3_data)

print("✅ Decryption complete! MP3 saved as 'decrypted_audio.mp3'")

✅ Decryption complete! MP3 saved as 'decrypted_audio.mp3'
